In [1]:
import os
import shutil
import zipfile
import urllib.request

def download_repo(url, save_to):
    zip_filename = save_to + '.zip'
    urllib.request.urlretrieve(url, zip_filename)
    
    if os.path.exists(save_to):
        shutil.rmtree(save_to)
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('.')
    del zip_ref
    assert os.path.exists(save_to)

In [2]:
REPO_PATH = 'LinearizedNNs-master'

download_repo(url='https://github.com/maxkvant/LinearizedNNs/archive/master.zip',
              save_to=REPO_PATH)

In [3]:
import sys
sys.path.append(f"{REPO_PATH}/src")

In [4]:
import time
import numpy as np
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torchvision.datasets import FashionMNIST


from pytorch_impl.estimators import SgdEstimator
from pytorch_impl.nns import Myrtle5, Myrtle7, Myrtle10
from pytorch_impl import ClassifierTraining
from pytorch_impl.matrix_exp import matrix_exp, compute_exp_term
from pytorch_impl.nns.utils import to_one_hot, print_sizes
from pytorch_impl.nns.primitives import Conv, Flatten, Normalize, ReLU2
from from_neural_kernels import to_zca, CustomTensorDataset, get_cifar_zca

In [5]:
device = torch.device('cuda:0') if (torch.cuda.is_available()) else torch.device('cpu')
device

device(type='cuda', index=0)

In [111]:
def matmul_via_torch(a, b, device, step=2560):
    n,  m = a.shape 
    m2, k = b.shape
    assert m == m2
    res = np.zeros([n, k], dtype=np.float32)
    
    for li in range(0, n, step):
        for lj in range(0, k, step):
            ri = min(n, li + step)
            rj = min(k, lj + step)
            
            a_row = torch.from_numpy(a[li: ri, :]).to(device)
            b_col = torch.from_numpy(b[:, lj: rj]).to(device)
            
            res[li: ri, lj: rj] = torch.matmul(a_row, b_col).cpu().numpy() 
    return res

In [112]:
np.linalg.norm([[1, 1], [1, 1]])

2.0

In [113]:
def compute_exp_term_numpy(M, device, n_iter=3):
    with torch.no_grad():
        n, _ = M.shape
        norm = np.sqrt(np.sum(M ** 2))
        steps = 0
        while norm > 1e-6:
            M /= 2.
            norm /= 2.
            steps += 1

        series_sum = np.zeros([n, n])
        prod       = np.eye(n)

        # series_sum: E + M / 2 + M^2 / 6 + ...
        for i in range(1, n_iter):
            series_sum += prod
            prod = matmul_via_torch(prod, M, device) / (i + 1)
        del prod
        
        exp = matmul_via_torch(M, series_sum, device) + np.eye(n)
        del M
        for step in range(steps):
            series_sum += matmul_via_torch(series_sum, exp, device)
            series_sum /= 2.
            exp = matmul_via_torch(exp, exp, device)
            print(series_sum[:3,:3])
            print(exp[:3, :3])
            print()
        return series_sum

In [10]:
kernels = np.load('../data/myrtle7_kernels.npz')
list(kernels)

['train_kernel', 'test_kernel', 'labels_train', 'labels_test']

In [56]:
a = np.eye(10)
a[0,1] = 1
b = np.eye(10)
matmul_via_torch(a, b, device)

array([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [62]:
compute_exp_term_numpy(- a, device)[:2,:2]

array([[ 9.99999999e-01, -9.31322575e-10],
       [ 0.00000000e+00,  9.99999999e-01]])

In [13]:
N = 16000
train_kernel = kernels['train_kernel'][:N, :N]
test_kernel  = kernels['test_kernel'][:, :N]

labels_train = kernels['labels_train'][:N]
labels_test  = kernels['labels_test']

In [14]:
train_kernel.shape, test_kernel.shape, labels_train.shape, labels_test.shape

((16000, 16000), (2000, 16000), (16000,), (2000,))

In [36]:
train_kernel[:5,:5]

array([[0.99999996, 0.99566936, 0.99371328, 0.99647623, 0.99540464],
       [0.99566936, 1.        , 0.99317682, 0.99668258, 0.99447368],
       [0.99371328, 0.99317682, 0.99999998, 0.99330106, 0.9941452 ],
       [0.99647623, 0.99668258, 0.99330106, 0.99999994, 0.99457984],
       [0.99540464, 0.99447368, 0.9941452 , 0.99457984, 1.        ]])

In [41]:
train_kernel[:5,:5]

array([[0.99999996, 0.99566936, 0.99371328, 0.99647623, 0.99540464],
       [0.99566936, 1.        , 0.99317682, 0.99668258, 0.99447368],
       [0.99371328, 0.99317682, 0.99999998, 0.99330106, 0.9941452 ],
       [0.99647623, 0.99668258, 0.99330106, 0.99999994, 0.99457984],
       [0.99540464, 0.99447368, 0.9941452 , 0.99457984, 1.        ]])

In [95]:
%%time 

sz = 16000

lr = 1e5

exp_term = - lr * compute_exp_term_numpy(- lr * train_kernel[:sz, :sz], device)

[[ 1.00000000e+00 -4.42165864e-11 -4.41297191e-11]
 [-4.42165864e-11  1.00000000e+00 -4.41058955e-11]
 [-4.41297191e-11 -4.41058955e-11  1.00000000e+00]]
[[ 1.00000000e+00 -8.84331417e-11 -8.82594070e-11]
 [-8.84331417e-11  1.00000000e+00 -8.82117599e-11]
 [-8.82594070e-11 -8.82117599e-11  1.00000000e+00]]

[[ 1.00000000e+00 -8.84331261e-11 -8.82593915e-11]
 [-8.84331261e-11  1.00000000e+00 -8.82117443e-11]
 [-8.82593915e-11 -8.82117443e-11  1.00000000e+00]]
[[ 1.00000000e+00 -1.76866159e-10 -1.76518689e-10]
 [-1.76866159e-10  1.00000000e+00 -1.76423395e-10]
 [-1.76518689e-10 -1.76423395e-10  1.00000000e+00]]

[[ 1.00000000e+00 -1.76866081e-10 -1.76518612e-10]
 [-1.76866081e-10  1.00000000e+00 -1.76423317e-10]
 [-1.76518612e-10 -1.76423317e-10  1.00000000e+00]]
[[ 1.00000000e+00 -3.53731818e-10 -3.53036880e-10]
 [-3.53731818e-10  1.00000000e+00 -3.52846292e-10]
 [-3.53036880e-10 -3.52846292e-10  1.00000000e+00]]

[[ 1.00000000e+00 -3.53731490e-10 -3.53036553e-10]
 [-3.53731490e-10  1.0

In [96]:
y_train = np.eye(10)[labels_train[:sz]] * 2 - 1

In [97]:
y_train[:5]

array([[-1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.],
       [-1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.],
       [-1.,  1., -1., -1., -1., -1., -1., -1., -1., -1.]])

In [98]:
exp_term[:5, :5]

array([[-6.75543266e+02,  5.87229182e-01,  2.78302637e+00,
        -1.62493695e+00,  7.92699184e-02],
       [ 5.87229184e-01, -7.31044861e+02,  1.21281686e-01,
         2.73982328e+00, -6.05988130e-01],
       [ 2.78302637e+00,  1.21281683e-01, -6.52939019e+02,
         5.42324090e-02, -1.94434468e+00],
       [-1.62493695e+00,  2.73982328e+00,  5.42324081e-02,
        -7.50512061e+02,  1.44969441e+00],
       [ 7.92699184e-02, -6.05988127e-01, -1.94434468e+00,
         1.44969441e+00, -6.87920425e+02]])

In [99]:
y_pred       = np.dot(test_kernel[:, :sz], np.dot(exp_term, - y_train))
np.average(y_pred.argmax(axis=1) == labels_test)

0.814

In [103]:
train_kernel = kernels['train_kernel']
test_kernel  = kernels['test_kernel']
labels_train = kernels['labels_train']
labels_test  = kernels['labels_test']

In [114]:
%%time 

sz = 50000

lr = 1e5

exp_term = - lr * compute_exp_term_numpy(- lr * train_kernel[:sz, :sz], device)

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 1024000000 bytes. Error code 12 (Cannot allocate memory)


In [115]:
y_pred       = np.dot(test_kernel[:, :sz], np.dot(exp_term, - y_train))
np.average(y_pred.argmax(axis=1) == labels_test)

ValueError: shapes (2000,50000) and (16000,10) not aligned: 50000 (dim 1) != 16000 (dim 0)